In [354]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation, Dropout

In [355]:
from ISLP import load_data
import numpy as np
import siuba
from siuba import _, if_else, select, mutate
from ISLP.models import (ModelSpec as MS,summarize)
from ISLP import confusion_table
import pandas as pd

In [356]:
smarket = load_data('Smarket')

In [357]:
smarket2=(
    smarket >>
    #mutate(Direction = if_else(_.Direction == 'Up',1,0)) >>
    mutate(train = if_else(_.Year < 2005,1,0) ) >>
    select(~_.Year,~_.Today)

)
train = (smarket2.train == 1)
# print(smarket2.head())

# print(design)

X = (
    smarket2 >>
    select(~_.train,~_.Direction)
)
y = smarket2['Direction']  == 'Up'

X_train, X_test = X.loc[train], X.loc[~train]
Y_train, Y_test = y.loc[train], y.loc[~train]
Y_test_raw =Y_test

In [358]:
X_train = (
    X_train >>    select(_.Lag1,_.Lag2)

)
X_test = (
    X_test >>    select(_.Lag1,_.Lag2)

)
X_train.columns

Index(['Lag1', 'Lag2'], dtype='object')

In [359]:
Y_test

998     False
999     False
1000    False
1001     True
1002    False
        ...  
1245     True
1246    False
1247     True
1248    False
1249    False
Name: Direction, Length: 252, dtype: bool

In [360]:
# X_train
feature_vector_length = 2
num_classes = 2

In [361]:
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Create the model


Feature shape: (2,)


In [362]:
Y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.

In [363]:
batch_size = 250
hidden_units = 2
dropout = 0.45

In [364]:
# best model
model = Sequential()
model.add(Dense(256, input_shape=input_shape))#, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(64))#, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(num_classes))
#Test results - Loss: 0.6916869282722473 - Accuracy: 0.5357142686843872%

In [365]:
# # best model
# model = Sequential()
# model.add(Dense(64, input_shape=2, activation='relu'))
# model.add(Dense(256, input_shape=input_shape, activation='relu'))
# model.add(Dense(64, input_shape=input_shape, activation='relu'))
# model.add(Dense(16, activation='relu'))

# model.add(Dense(num_classes, activation='softmax'))
# #Test results - Loss: 0.7014935612678528 - Accuracy: 0.5595238208770752%

In [366]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=15, batch_size=batch_size, verbose=1, validation_split=0.2)

# Test the model after training
test_results = model.evaluate(X_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

Epoch 1/15
4/4 [==============================] - 2s 565ms/step - loss: 4.3724 - accuracy: 0.5038 - val_loss: 1.5724 - val_accuracy: 0.4800
Epoch 2/15
4/4 [==============================] - 0s 13ms/step - loss: 3.4533 - accuracy: 0.5238 - val_loss: 0.8609 - val_accuracy: 0.4950
Epoch 3/15
4/4 [==============================] - 0s 14ms/step - loss: 2.5454 - accuracy: 0.4674 - val_loss: 0.6982 - val_accuracy: 0.5100
Epoch 4/15
4/4 [==============================] - 0s 14ms/step - loss: 2.2224 - accuracy: 0.5263 - val_loss: 0.7076 - val_accuracy: 0.5050
Epoch 5/15
4/4 [==============================] - 0s 12ms/step - loss: 2.0679 - accuracy: 0.4862 - val_loss: 0.6960 - val_accuracy: 0.5650
Epoch 6/15
4/4 [==============================] - 0s 11ms/step - loss: 1.9373 - accuracy: 0.5025 - val_loss: 0.7784 - val_accuracy: 0.5500
Epoch 7/15
4/4 [==============================] - 0s 13ms/step - loss: 1.6451 - accuracy: 0.4687 - val_loss: 0.7016 - val_accuracy: 0.5600
Epoch 8/15
4/4 [==========

In [367]:
test_results

[0.683149516582489, 0.5396825671195984]

In [368]:
test_pred_raw = model.predict(X_test)
test_pred = np.argmax(test_pred_raw, axis=1)
test_pred


8/8 [==============================] - 0s 3ms/step


array([1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1], dtype=int64)

In [369]:
# print(np.array(Y_test))
# print(test_pred)
# print(len(np.array(Y_test)))
# print(len(test_pred))
df1 = pd.DataFrame(
   
    {'pred':  test_pred}
    
)

df2 = (df1 >>
       mutate(y=np.array(y.loc[~train])) >>
       mutate(y=if_else(y,1,0))
)

df2.head()


,pred,y
0,1,1
1,1,1
2,1,0
3,1,1
4,1,1


In [370]:
ct = confusion_table(test_pred, Y_test_raw)
print(ct)

Truth       0   1
Predicted        
0          57  62
1          54  79


In [371]:
type(ct)

pandas.core.frame.DataFrame

In [372]:
TN = ct.iloc[0,0]
TP = ct.iloc[1,1]
FN = ct.iloc[0,1]
FP = ct.iloc[1,0]
Accuracy = (TP + TN) /(TN+TP+FN+FP)
Recall = TP /(TP+FN)
Precision = TP/(TP+FP)
F_score = Recall*Precision/(Recall +Precision)
df_r = pd.DataFrame(
 {'Accuracy': [Accuracy]
  ,  'Precision':[Precision],   'Recall':[Recall],    'F_score':[F_score]
 }
)
df_r

,Accuracy,Precision,Recall,F_score
0,0.539683,0.593985,0.560284,0.288321
